# Masked-ICA pipeline
Niall Bourke ~ April 2019  
Based on Sara's paper https://academic.oup.com/brain/article/141/1/148/4654726


### If doing batch processing with output that may be usful to others -> direct to c3nl_processed in sub dir.
#### The first HCP pre-processing steps shouldnt have to be repeated and the pipeline can begin from mICA.
#### * Do not write to original HCP directories!!   
  
---------

### Steps:
#### 1. Set environment
#### 2. HCP pre-processing
#### 3. mICA for HCP data (add mask of interest)
#### 4. Pre-processing of test data
#### 5. Applying network masks from HCP to test data



### Overview: 
You want to create your components with unsmoothed independent data (using an anatomical mask of interest). For this we will use the Human Connectome Projects 100 unrelated subjects. After selecting our components we will run out stats on our test data using smoothed data. 

##### Two data sets
1. Independant control group from the HCP to create functional connectivity components
2. Test set containing two groups of controls and paitents

*These datasets will be treated slightly differently. The components should be dervived from unsmoothed data while the test set should have fully processed and smoothed stata ready for statistics. 

##### Independant set:
    1. subject level melodic
    2. mICA preprocessing
    3. Group ICA
    3. Component selection

##### Test set:
    1. Single subject melodic
    2. QA: framwise displacement (before ICA-AROMA) with a threshold
    3. Fix/ICA-AROMA
       With Fix/manual do all cleaning first time round. Alternatvily if running ICA AROMA do first level feat without temporal filtering - check documentation - ICA-AROMA will produce a text file with  suggested labels for noise/components. This needs to be reviewed and edited where necessary and then fed back into the second stage. 
    4. Dual regression on grey matter using network mask (can also included motion regressors)


##### Network analysis
* When the components have been got - do a dual regression selecting significant regions constrained to GM
Remove components that are noise or have high overlap with other components (quant approach)


* To process your own data do;
single subject melodic , then fix

* randomise stage
dual regression script and add network mask created from using a component to do whole brain connectivity.  

* reproducibility analysis 
* crib sheet for ICA (kelly, 2008)
* fslreg_filt


## Initial setup
Bash variables dont carry across cells anymore. Here I have saved them to a text file, which can then be read at the start of a cell. 

In [10]:
import os

project = "caudalACC" # "rostalACC" # "lesion"   

directory = ("/rds/general/user/nbourke/ephemeral/" + project)
if not os.path.exists(directory):
    os.makedirs(directory)
    
raw = ("/rds/general/project/c3nl_djs_imaging_data/live/data/raw/" + project + "/")
source = ("/rds/general/project/c3nl_djs_imaging_data/live/data/sourcedata/")
workingDir = ("/rds/general/user/nbourke/ephemeral/mica/" + project )
setup = (workingDir + "/setup.sh" )

Can the above set variables in python be exported to bash file below??

In [11]:
%%writefile $setup

$(: Project label) 
project="caudalACC" # "rostalACC" # "lesion" 

$(:  Where is the RAW directory?)
export rawDir=/rds/general/user/nbourke/home/projects/${project}/raw;
export testSubjects=/rds/general/user/nbourke/home/projects/${project}/scripts/rawSubj.txt

$(: dependencies)
export dep=/rds/general/project/c3nl_shared/live/dependencies/
export templates=/rds/general/user/nbourke/home/templates
export workingDir=/rds/general/user/nbourke/ephemeral/mica/${project}

$(: HCP paths)
export hcpDir=/rds/general/project/c3nl_djs_imaging_data/live/HCP100
export subjects=/rds/general/project/c3nl_djs_imaging_data/live/HCP100/subj.txt


# Define modules    
fsl="module load fsl";
micaEx="export MICADIR=/rds/general/project/c3nl_shared/live/dependencies/mICA_Toolbox";
micaDir=/rds/general/project/c3nl_shared/live/dependencies/mICA_Toolbox;


Overwriting /rds/general/user/nbourke/ephemeral/mica/caudalACC/setup.sh


In [12]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir

/rds/general/user/nbourke/ephemeral/mica/caudalACC


## Prep the HCP data for analysis

* Functional is already in MNI 2mm space  
* Skull strip T1 (Done in HCP pipeline)
* Register T1_brain to MNI2mm space
      

### Quick look over the dimensions of the data

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir

module load fsl
module load mrtrix 
#----------------------------


## Loop over all subjects ##
for subj in 100307; #`cat $subjects`;
do 
    ## Set data to process ##
    T1=${hcpDir}/struct/${subj}/T1w/T1w_acpc_dc_restore.nii.gz
    T1Brain=${hcpDir}/struct/${subj}/T1w/T1w_acpc_dc_restore_brain.nii.gz
    restClean=${hcpDir}/rest/${subj}/MNINonLinear/Results/rfMRI_REST1_LR/rfMRI_REST1_LR_hp2000_clean.nii.gz
    
    T1_2mm=struct/100307/MNINonLinear/T1w_restore.2.nii.gz
    
    template=/apps/fsl/5.0.10/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz
    
    echo "Looking at the data"
    echo ""
    echo "T1 is $T1"
    mrinfo $T1
    echo "Brain is $T1Brain"
    mrinfo $T1Brain
    echo "rest is $restClean"
    mrinfo $restClean
    echo "template is $template"
    mrinfo $template
    
done


#### The HCP has the resting data in MNI2mm space along with the T1. The T1_brain however needs to be registered to the same space. 

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir

module load fsl
#----------------------------

hcpDir=/rds/general/project/c3nl_djs_imaging_data/live/HCP100
subjects=${hcpDir}/subj.txt
mkdir -p ${EPHEMERAL}/mica
workingDir=${EPHEMERAL}/mica

module load fsl

## Set empty Command File ##
echo "" > ${workingDir}/HCP_Reg_job.txt

## Loop over all subjects ##
for subj in `cat $subjects`;
do 
    ## Set data to process ##
    #T1=${hcpDir}/struct/${subj}/T1w/T1w_acpc_dc_restore.nii.gz
    T1Brain=${hcpDir}/struct/${subj}/T1w/T1w_acpc_dc_restore_brain.nii.gz
    #restClean=${hcpRest}/${subj}/${subj}.nii.gz
    template=/apps/fsl/5.0.10/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz
   
    # Only need to register T1_brain to 2mm space
    echo "module load fsl; flirt -in ${T1Brain} -ref ${template} -out ${hcpDir}/struct/${subj}/T1w/MNI152_2mm_T1w_acpc_dc_restore_brain.nii.gz;" >> ${workingDir}/HCP_Reg_job.txt
done

## Run jobs ## 

    /rds/general/project/c3nl_shared/live/dependencies/hpcSubmit ${workingDir}/HCP_Reg_job.txt 01:00:00 1 8Gb
    # 1. submission command, 2. jobFile, 3. Requested time, 4. number of cpus, 5. ammount of RAM

------------

# Masked ICA toolbox

This is in the dependency folder on the cluster but is has sparse documentation. It uses fsl native commands such as melodic. 
    
* Initially this can just be run through the gui on one subject to make sure its doing what you want (locally not on cluster). When it is running the command output will be printed to the screen. This can be taken and adjusted to batch run a load of subjects.

steps:
    1. Pre-processing
    2. Group ICA
    3. Dual regression
    4. Component selection


## 1. pre-processing 100-HCP (unsmoothed cleaned resting state) participants

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir

module load fsl
#----------------------------

#  mkdir -p ${EPHEMERAL}/mica/${project}
#  workingDir=${EPHEMERAL}/mica/${project}

# Set empty command File 
echo -n "" > ${workingDir}/${project}_preproc_job.txt
echo -n "" > ${workingDir}/HCP_rest_subj_path.txt

# Get list of subjects
for subj in `cat $subjects`; do  
    ls ${hcpDir}/rest/${subj}/MNINonLinear/Results/rfMRI_REST1_LR/rfMRI_REST1_LR_hp2000_clean.nii.gz >> ${workingDir}/HCP_rest_subj_path.txt
done
   
# Set job variables
input=${workingDir}/HCP_rest_subj_path.txt
mkdir -p ${workingDir}/rest
outputDir=${workingDir}/rest
#mask=${templates}/lesionMask2mm.nii.gz
mask=${templates}/ACC_masks/${project}_MNI2mm.nii.gz
template=/apps/fsl/5.0.10/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz

# -crop

#~~~~~~~~~~~~~~~~~~~~~#
# Run job
#~~~~~~~~~~~~~~~~~~~~~#

# Set job in textfile for submission wrapper
echo "${fsl}; ${micaEx}; ${micaDir}/bin/mica_preproc $input $outputDir -smooth 3.0 -mask $mask -bgimage $template" >> ${workingDir}/${project}_preproc_job.txt
job=${workingDir}/${project}_preproc_job.txt

    ## Run job
    ${dep}/hpcSubmit ${job} 32:00:00 1 12Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    cat ${job}

## 2. Group ICA

The below cell generated the melodic commands using mica - this was serial so, I broke it up and paralleised it below. 

In [ ]:
# %%script bash

# #~~~~~~~~~~~~~~~~~~~~~#
# # Set paths & tools
# #~~~~~~~~~~~~~~~~~~~~~#

# project=`awk '{if(NR==1) print $0}' MICAsetup.txt`
# dep=`awk '{if(NR==2) print $0}' MICAsetup.txt`
# templates=`awk '{if(NR==3) print $0}' MICAsetup.txt`
# workingDir=`awk '{if(NR==4) print $0}' MICAsetup.txt`

# fsl=`awk '{if(NR==5) print $0}' MICAsetup.txt`
# micaEx=`awk '{if(NR==6) print $0}' MICAsetup.txt`
# micaDir=`awk '{if(NR==7) print $0}' MICAsetup.txt`

# hcpDir=`awk '{if(NR==8) print $0}' MICAsetup.txt`
# subjects=`awk '{if(NR==9) print $0}' MICAsetup.txt`

# #~~~~~~~~~~~~~~~~~~~~~#
# # Set variables
# #~~~~~~~~~~~~~~~~~~~~~#

# # Set empty command File 
# echo -n "" > ${workingDir}/${project}_ICA_job.txt
 
# # Set command variables
# input=${workingDir}/rest/input_preproc.txt # Should be created from previous step
# mkdir -p ${workingDir}/ICA
# outputDir=${workingDir}/ICA 
# mask=${workingDir}/rest/final_mask.nii.gz 
# template=${workingDir}/rest/bgimage.nii.gz

# ## Set job

# echo "${fsl}; ${micaEx}; ${micaDir}/bin/mica $input $outputDir -dim 2-12 -mask $mask -merge-stats -parcel -extra --bgimage=$template --report --Ostats" >> ${workingDir}/${project}_ICA_job.txt   
# job=${workingDir}/${project}_ICA_job.txt
# #

#     # Run job
#     ${dep}/hpcSubmit ${job} 62:00:00 1 12Gb
#     echo ""; echo "***"; echo ""; echo "Submitted commands:"
#     cat ${job}

Reproducability analysis 

### Starting Melodic ###

No specified dimensions will take longer and has been failing..
It took ~55hrs to run 100 subjects with a mask on the dim0 component (98 components came out)
* Was run wiith unsmoothed data - likely to be quicker with 6mm smoothing. 
* dim 0 timed out with dual regression for 62hrs but with smoothing looks like will compleate. 

### Generate ICA-based Parcellation ###

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir

module load fsl
#----------------------------


mkdir -p ${workingDir}/ICA
outputDir=${workingDir}/ICA 

# Set empty command File 
 echo -n "" > ${workingDir}/${project}_ICA_job.txt

### Starting Melodic ###

# dim0  
#echo "/apps/fsl/5.0.10/fsl/bin/melodic -i $workingDir/rest/input_preproc.txt -o ${outputDir}/dim0_run2 --mask=${workingDir}/rest/final_mask.nii.gz --bgimage=${workingDir}/rest/bgimage.nii.gz --report --Ostats" >> ${workingDir}/${project}_ICA_job.txt     

# dim 2-14
for ii in {2..12};
do
   echo "/apps/fsl/5.0.10/fsl/bin/melodic -i ${workingDir}/rest/input_preproc.txt -o ${outputDir}/dim${ii} --mask=${workingDir}/rest/final_mask.nii.gz --dim=${ii} --bgimage=${workingDir}/rest/bgimage.nii.gz --report --Ostats" >> ${workingDir}/${project}_ICA_job.txt     
done

job=${workingDir}/${project}_ICA_job.txt
#

    # Run job
    ${dep}/hpcSubmit ${job} 62:00:00 1 12Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    cat ${job}


### Cross correlation of components to check spatial overlap
Go to stats dir generated with melodic above - Merge the thresholded Z maps into two 4D files and compare 

In [ ]:
fslmerge -t all_threshZ_A `ls thresh*`
fslmerge -t all_threshZ_B `ls thresh*`

fslcc all_threshZ_A all_threshZ_B

### Spatially smooth cleaned data for input into dual-regression
* Don't need to rerun (unless altering smoothing kernal)

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir

module load fsl
#----------------------------
# Set empty command File 
echo -n "" > ${workingDir}/${project}_smooth_job.txt

# Job
ix=`cat $subjects`
for subj in $ix; do
    input=${hcpDir}/rest/${subj}/MNINonLinear/Results/rfMRI_REST1_LR/rfMRI_REST1_LR_hp2000_clean.nii.gz
    output=${hcpDir}/rest/c3nl_processed/${subj}/rfMRI_REST1_LR_6mm_gauss
    mkdir -p $output
    echo "${fsl}; fslmaths $input -kernel gauss 2.549987 -fmean $output/rfMRI_REST1_LR_hp2000_clean_6mm_gauss.nii.gz" >> ${workingDir}/${project}_smooth_job.txt
done

job=${workingDir}/${project}_smooth_job.txt

    # Run job
    ${dep}/hpcSubmit ${job} 02:00:00 1 12Gb
    echo ""; echo "***"; echo ""; echo "Head submitted commands:"
    head ${job}

## 3. Dual-regression (inc. grey matter mask)

Take melodic ICA output of the earlier step for each component, and use this to drive dual regression which back projects onto individual data (This should be done on smoothed input [6mm]). This is done in three steps:  


**Timecourses at subject level**
> For each subject, apply the spatial (sub)component masks and thus derive (regress) average time series.

**Spatial maps at subject level**
> For each timeseries derived above, identify (regress) whole brain spatial map of areas of co-activation.

**Permutation testing**
> 
--  

May also run faster after smoothing.. 

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir

module load fsl
#----------------------------


# Set empty command File 
echo -n "" > ${workingDir}/${project}_dual_job.txt

echo -n "" > ${workingDir}/6mm_subj.txt
for ii in `cat $subjects`; do
   ls ${hcpDir}/rest/c3nl_processed/${ii}/rfMRI_REST1_LR_6mm_gauss/* >> ${workingDir}/6mm_subj.txt
done

# Command input variables
subjects=${workingDir}/6mm_subj.txt
ROI_mask=${templates}/ACC_masks/${project}_MNI2mm_mask.nii.gz #ROI_mask=${templates}/lesionMask2mm.nii.gz #
brain_mask=${templates}/MNI152_T1_2mm_brain_pve_1_mask.nii.gz # GM masking
ICAdir=${workingDir}/ICA

# Settings: 
des_norm=1; # whether to variance normalise the time courses used in the stage 2 regression
designmat=-1; # specify we are doing a one sample t test model **CHECK 
designcon=-1;
designfts=0;
designgrp=0;
n_perm=5000 #1; # specify number of permutations for randomise


# Command
for directory in `ls -d $ICAdir/*/` ;
do
    dim=`basename $directory`;
    groupICA=${ICAdir}/${dim}/melodic_IC.nii.gz
    output=$workingDir/${projectLabel}/DR6mm/pve1Masked_${dim}_dual_regression
    # ${designcon} # removed as $designmat is set to -1
    echo "${fsl}; ${micaEx}; ${micaDir}/bin/dual_regression_roi ${groupICA} ${des_norm} ${ROI_mask} ${brain_mask} ${designmat} ${designfts} ${designgrp} ${n_perm} ${output} -in-step1 \`cat $subjects\` -in-step2 \`cat $subjects\`" >> ${workingDir}/${project}_dual_job.txt;      

done

job=${workingDir}/${project}_dual_job.txt

    # Run job
    ${dep}/hpcSubmit ${job} 68:00:00 1 12Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    head ${job}
    
# Usage: dual_regression <group_IC_maps> <des_norm> <step1_mask> <step2_mask> <design.mat> <design.con> <design.fts> <design.grp> <n_perm> <output_directory> -in-step1 <input1> <input2> ... -in-step2 <input1> <input2> ...
# e.g.   dual_regression groupICA.gica/groupmelodic.ica/melodic_IC 1 design.mat design.con 500 grot \`cat groupICA.gica/.filelist\`

## 4. Component Selection:  
Have a reasonable amount of components you want to use, based on previous literature, after removing noise. Or quantitative analysis from generate multiple components.   

Worth looking at how components vary and what regions share common variance 

    fslmaths
    -Tmean
    -inverse warp


-------------

# Reproducibility Analysis
(notes from Richard)

The approach detailed in mICA produces an estimate for ideal dimensionality for decomposition. This is necessary because: the underlying dimensionality is not known in fMRI data, different instantiations (realisations) of ICA for the same dimensionality produce different components, and the stability of components will differ across dimensionalities. The approach is as below:

The following discussion was helpful to developing these cells: https://www.nitrc.org/forum/message.php?msg_id=24083

Permutation of group membership
> The group is split in 2 in different ways, for N permutations. Each permutation is then subjected to group mICA.

Exploration of dimensionalities
> The above process is done across a range of dimensionalities.

Minimising temporal cross-correlation
> iccorr.py uses fslcc to produce 'spatial' cross-correlations between group 1 and group 2. Within each dimensionality, across permutations, cross-correlations are calculated. Negative cross-correlations are removed, then the result is subtracted from 1 to give a 'cost'. 'Cost' is high where components are highly independent, and low when they are highly related. This cost matrix is then sorted, such that components are aligned - so the diagnoal of the matrix represents the cost for two equivalent components; again if they are more similar, the cost is low (high reproducibility). The mean of this diagonal is then taken as a measure of the cost or this dimension and permutation. 

> Mean 'cost' is thus the mean for a dimension across permutations, and a lower value corresponds to greater reproducibility.

Practically, this process is computationally intensive. Given the strengths of the HPC (array computing), and that the number of permutations will likely exceed the dimensionality count, the goal is to run D dimensionality jobs each with N permutations.

## Running the split group reproducability analysis
### STEP 1.

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir

module load fsl
ICAAROMA=~/c3nl_tools/ICA-AROMA-master/ICA_AROMA.py
aroma="source activate aroma"
#----------------------------


# Set empty command File 
echo -n "" > ${workingDir}/${project}_reproducability_job.txt

# run splithalf
reproducibility_permutations=50
reproducibility_dimensions="2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50";
mkdir -p $workingDir/reproducibility_dir
reproducibility_dir=$workingDir/reproducibility_dir

echo "${aroma}; ${micaEx}; python ${micaDir}/py/splithalf.py $workingDir/rest/input_preproc.txt ${reproducibility_permutations} ${reproducibility_dir}" >> ${workingDir}/${project}_reproducability_job.txt

job=${workingDir}/${project}_reproducability_job.txt

    # Run job
    ${dep}/hpcSubmit ${job} 71:00:00 1 12Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    cat ${job}
    

### STEP 2.

Loop over the 50 samples in each of the split groups 1 & 2 -> Run x dim for each. 

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir

module load fsl
ICAAROMA=~/c3nl_tools/ICA-AROMA-master/ICA_AROMA.py
aroma="source activate aroma"
#----------------------------

# Set empty command File 
echo -n "" > ${workingDir}/${project}_bigICA_job.txt

mkdir -p ${workingDir}/ICA
outputDir=${workingDir}/ICA 


### Starting Melodic ###
for group in {1..2};
    do
    for sample in `ls $workingDir/reproducibility_dir`;
        do
        for dim in {2..50};
            do
            echo "${fsl}; /apps/fsl/5.0.10/fsl/bin/melodic -i $workingDir/reproducibility_dir/${sample}/group${group}_input.txt -o $workingDir/reproducibility_dir/${sample}/group${group}/dim${dim} --mask=$workingDir/rest/final_mask.nii.gz --dim=${dim} --bgimage=$workingDir/rest/bgimage.nii.gz --report --Ostats" >> ${workingDir}/${project}_bigICA_job.txt    
        done
    done
done

job=${workingDir}/${project}_bigICA_job.txt

    # Run job
    ${dep}/hpcSubmit ${job} 24:00:00 1 12Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    head ${job}


### Step 3 - running the reproducibility analysis

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir

module load fsl
ICAAROMA=~/c3nl_tools/ICA-AROMA-master/ICA_AROMA.py

# aroma="source activate aroma" # missing munkes package, could just install
aroma="source activate py27"
#----------------------------

# /share/apps/fsl/bin/fsl_sub -N mICASplitHalf_step3 -T 30 -l /group/HCP/analysis/SaraStriatum/test_reprodcommand/tmp_log python /share/apps/mICA/mICA_Toolbox/py/ic_corr.py /group/HCP/analysis/SaraStriatum/test_reprodcommand 50 2-20

mkdir -p ${workingDir}/reproducibility_dir
reproducibility_permutations=50
reproducibility_dimensions="2-50";

echo "${aroma}; ${micaEx}; python ${micaDir}/py/ic_corr.py ${workingDir}/reproducibility_dir ${reproducibility_permutations} ${reproducibility_dimensions} 1" > ${workingDir}/${project}_step3_RA_job.txt

    job=${workingDir}/${project}_step3_RA_job.txt
    # Run job
    ${dep}/hpcSubmit ${job} 42:00:00 1 12Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    head ${job}

## Validate ICA result at chosen dimensionality - Spatial and Temporal Correlation, and Spatial Similarity
As per De Simoni et al. 2018, if ICA has been effective, then:
1. Spatial correlation will be low between components within ROI mask.
2. Also Spatial similarity (Dice) should be low.
2. Finally temporal correlation will also be low.

Practically, for now, I opt to look at spatial similarity (Dice co-efficient), and temporal cross-correlation using ```fslcc``.

### Spatial Correlation of Components
Pearson's Spatial Correlation defined as in https://www.jiscmail.ac.uk/cgi-bin/webadmin?A2=fsl;1057accd.1202
The result will be a matrix (csv) of maps vs. maps, with inter-component spatial similarity as the value.

In [ ]:
%%bash 

dims_stats=20

for dim in ${dims_stats};
do

    echo "Preparing environment variables and folders for dimension $dim";
    
    $(: Subfolder to save results in)
    export run=maskthresh_${mask_thresh}_ROP2_GM${gmseg_thresh_pct}_dim_${dim}_drGMmask;
    echo mICA will be save to subfolder \$run;
    
    $(: mICA results folders)
    export mica_group_ica=${ANALYSISDIR}/mica_group_ica_\${run};
    export mica_dual_regression=${ANALYSISDIR}/mica_dual_regression_\${run};
    check_exists \${mica_group_ica} \"mICA group ica folder set\";
    check_exists \${mica_dual_regression} \"mICA dual regression folder set\";   
    results_folder=\${mica_dual_regression};

    $(: where results are saved)
    csv=$ANALYSISDIR/component_spatial_correlation_\${run}.csv;
    
    
    echo "One minus p setting is $oneminusp" > $ANALYSISDIR/component_spatial_correlation_settings_\${run}.txt;
    echo "Results will be saved to \$csv";
   
    $(:  where intermediate files go)
    components_folder=${inputs_folder}/component_spatial_correlation_\${run};
    
    $(: where is the melodic file)
    melodic_file=\${mica_group_ica}/dim${dim}/melodic_IC.nii.gz;


    check_exists \${components_folder} \"Created dice folder\";
    rm \${components_folder}/components*;
    
    $(: Extract components to components folder)
    
    fslsplit \${melodic_file} \${components_folder}/components;
    
    echo Producing a list of components in \${components_folder};
    cd \${components_folder};
    
    components=\$(ls);
    components_array=(\$components);
    num_components=\$(echo \$components | wc -w);
    echo \${num_components} components found;
    names=\"\";
    names_header=\"\";
    
    $(: extract names)
    echo Getting component names;
    for i in \$(seq 1 \${num_components});
    do
        iminus1=\$((\$i - 1 ));
        tempname=\`echo \${components_array[\$iminus1]} | grep -o -P '(?<=components).*(?=.nii.gz)'\`;
        names=\"\${names} \${tempname}\";
        names_header=\"\${names_header},\${tempname}\";
    done;
    names_array=(\$names);
    

    $(: write csv header)
    echo \${names_header}>\${csv};

    cd \${components_folder};
    $(: for each component, for each other component, calculate spatial similarity between i and j)
    $(: Let a is component file for i, and b be component file for j, A is volume of a, B is volume of b, AnB is the intersection of A and B)
    $(:  So dice D is 2xAnB div A + B)
    
    
    echo Working through each component row;
    for i in \$(seq 0 \$((\${num_components} - 1)));
    do
        $(: row header)
        row=\"\";
        row_header=\${names_array[\${i}]};
        row=\${row_header};
        a=\${components_folder}/\${components_array[\$i]};
        $(: mask )
        MA=$(fslstats \$a -k $mask -m);
        $(: demeaned and sqr root demeaned)
        demeaned_a=${inputs_folder}/a_\${names_array[\$i]}_demeaned.nii.gz;
        fslmaths \${a} -sub \$MA -mas $mask \${demeaned_a};
        demeaned_sqr_a=${inputs_folder}/a_\${names_array[\$i]}_demeaned_sqr.nii.gz;
        fslmaths \${demeaned_a} -sqr \${demeaned_sqr_a};
        den_a=\$(fslstats \${demeaned_sqr_a} -k $mask -m);


        echo Component \${names_array[\$i]} has volume \$A;
        for j in \$(seq 0 \$((\${num_components} - 1)));
        do

            b=\${components_folder}/\${components_array[\$j]};
            $(: mask )
            MB=$(fslstats \$b -k $mask -m);
            $(: demeaned and sqr root demeaned)
            demeaned_b=${inputs_folder}/b_\${names_array[\$j]}_demeaned.nii.gz;
            fslmaths \${b} -sub \$MB -mas $mask \${demeaned_b};
            demeaned_sqr_b=${inputs_folder}/b_\${names_array[\$j]}_demeaned_sqr.nii.gz;
            fslmaths \${demeaned_b} -sqr \${demeaned_sqr_b};
            den_b=\$(fslstats \${demeaned_sqr_b} -k $mask -m);

           $(: more work)

            demeaned_prod=${inputs_folder}/prod_\${names_array[\$i]}_\${names_array[\$j]}_demeaned.nii.gz;
            fslmaths \${demeaned_a} -mul \${demeaned_b} \${demeaned_prod};
            num=\$(fslstats \${demeaned_prod} -k $mask -m);
            denprod=\$(echo \"scale=${bc_scale}; sqrt(\${den_a}*\${den_b})\" | bc -l);
            true_r=\$(echo \"scale=${bc_scale}; \${num}/\${denprod}\" | bc -l);
            
             $(: Pearson)
           
            $(: update row)
            row=\"\${row},\${true_r}\";
                   
            
        done;
        $(: add row to csv)
        echo \${row} >> \$csv;
    done";
                    
    echo ${inputcommands//$'\n'/ }>>$notebookpath/hcpvestibular_component_spatial_correlation${randref}.txt;

echo Finished for dimension $dim with results in $csv;
done;
                      
                      
cd $notebookpath/logs;
$notebookpath/dependencies/phd_hpc_submit.sh "$notebookpath/hcpvestibular_component_spatial_correlation${randref}.txt" 0:30:00 1 2Gb $notebookpath/hcpvestibular_fourth${randref}_${dim}.txt


echo All done with spatial similarity for all dimensions.

-------------




# Testing dataset:

## Pre-processing resting state data run with rest_preproc.ipynb notebook

This involves:
1. Setup dependencies & project directories
2. Skull stripping using BET in FSL
3. Lower level FEAT (without high-pass filtering because of ICA AROMA) - the preprocessing tabs
        - motion correction
        - slice timing correction
        - spatial smoothing
        - registration
4. Create functional masks
5. Run matlab script to calculate 24 motion parameters = .txt file to include in stage 8 (ORDER OF THIS??)
6. Identifying motion, artifacts, noise components 
        5a. ICA AROMA
        5b. Motion outliers   
7. Denoising using components
8. Regress out white matter, CSF and motion paramters (6 that came from MCFLIRT +18)
9. Lower level FEAT of cleaned data (with high-pass filtering) - the preprocessing tabs

# Analysis of filtered func clean data

### Binerising networks from HCP data for use as masks 

In [45]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir

module load fsl
#----------------------------
#workingDir=/rds/general/user/nbourke/home/projects/micaOutput/rostalACC

drDir=${workingDir}/DR6mm/pve1Masked_dim12_dual_regression # $workingDir/DR6mm/pve1Masked_dim12_dual_regression # 
groupICA=${ICAdir}/dim12/melodic_IC.nii.gz # $workingDir/ICA/dim12/melodic_IC.nii.gz   #


for dim in $(seq -f "%02g" 0 11)
do

    #options
    network=${drDir}/dr_stage3_ic00${dim}_tfce_corrp_tstat1.nii.gz
    threshold=0.95
    maskOut=$workingDir/networkMasks
    mkdir -p $maskOut
    
    #Command
    fslmaths $network -thr $threshold -bin ${maskOut}/IC_${dim}_mask

done


/rds/general/user/nbourke/ephemeral/mica/lesion
00
01
02
03
04
05
06
07
08
09
10
11


 /home/nbourke/anaconda3/envs/py27/bin /apps/gcc/6.2.0/ /opt/ibutils/bin /rds/general/user/nbourke/home/anaconda3/bin /apps/mrtrix/3.0/bin /apps/ants/2015-02-23/bin/bin /apps/gcc/6.2.0/bin /usr/local/sbin /opt/pbs/bin /usr/lib64/qt-3.3/bin /apps/ants/2015-02-23/bin/ /rds/general/user/nbourke/home/perl5/bin /apps/anaconda3/4.5.12/install


### Dual-regression (back projection) of HCP defined components to a test data set
##### (could include neuropsych in design matrix)

1. Select dimension from HCP data to run
2. Choose components to run from this.
3. For each component from the HCP apply spatial & temporal back projection (melodic_IC) to new test dataset using a HCP mask for that network


##### Caudal ACC
0 - keep  
1 - ?  
2 - keep  
3 - ?  
4 - keep (nosiy DMN)  
5 - x  
6 - keep  
7 - keep (cleaner DMN)  
8 - ACC  
9 - keep subcortical  
10 - x  
11 - x  

#### Pre-processed test set
* Collate tesing data subjects for the following steps

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir

echo "" > ~/projects/squeezy/derivatives/con_ffdc.txt

for ii in `ls -d ~/projects/squeezy/derivatives/controlTestSet/*/`;
do
    subj=`basename $ii`
    fullpath=${ii}/func/rest/postICA.feat/filtered_func_data_clean_standard.nii.gz 
    echo $fullpath >> ~/projects/squeezy/derivatives/con_ffdc.txt
done

#ls ~/eph/DREAM_REST_clean/* >> ~/projects/squeezy/derivatives/dream_ffdc.txt
# At some point I manually selected the patients of interest from this list

### Testing group dual regression with derived network masks
* Why is there no output for this - did it fail?  
* Paths to data have now changed 

In [13]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir

#jobfile
echo "" > ${workingDir}/${project}_testSet_analysis_job.txt;  
job=${workingDir}/${project}_testSet_analysis_job.txt;  

# -- Specify Input --#

testDir=$workingDir/test
mkdir -p ${testDir}
cp $workingDir/ICA/dim12/melodic_IC.nii.gz ${testDir}

# Options
IC_map=${testDir}/melodic_IC.nii.gz # This is the HCP melodic (components defined with this)
input=~/projects/squeezy/scripts/subPaths.txt
brain_mask=${templates}/MNI152_T1_2mm_brain_pve_1_mask.nii.gz # GM masking
des_norm=1; # whether to variance normalise the time courses used in the stage 2 regression
design_mat=~/projects/squeezy/scripts/design/basic.mat #change path
design_con=~/projects/squeezy/scripts/design/contrast.con # change path
designfts=0;
designgrp=0;
n_perm=5000


# Loop over networks to back project with
for dim in $(seq -f "%02g" 0 11);
    do
    HCP_mask=$workingDir/networkMasks/IC_${dim}_mask.nii.gz
    output=${testDir}/randomise_output/dim${dim}
    mkdir -p $output
    
    # dual regression
    echo "${fsl}; ${micaEx}; ${micaDir}/bin/dual_regression_roi ${IC_map} ${des_norm} ${HCP_mask} ${brain_mask} ${design_mat} ${design_con} ${designfts} ${designgrp} ${n_perm} ${output} -in-step1 \` cat $input\` -in-step2 \` cat $input\`" >> ${job} ;      
    done
    
    # Run job
    ${dep}/hpcSubmit ${job} 36:00:00 1 12Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    head ${job}

/rds/general/user/nbourke/ephemeral/mica/caudalACC
input is = /rds/general/user/nbourke/ephemeral/mica/caudalACC/caudalACC_testSet_analysis_job.txt
Walltime = 36:00:00
Number of CPUs = 1
Memory = 12Gb
Array jobs submitted: 13
Job submitted: Mon 20 Jan 17:56:24 GMT 2020
979305[].pbs

***

Submitted commands:

module load fsl; export MICADIR=/rds/general/project/c3nl_shared/live/dependencies/mICA_Toolbox; /rds/general/project/c3nl_shared/live/dependencies/mICA_Toolbox/bin/dual_regression_roi /rds/general/user/nbourke/ephemeral/mica/caudalACC/test/melodic_IC.nii.gz 1 /rds/general/user/nbourke/ephemeral/mica/caudalACC/networkMasks/IC_00_mask.nii.gz /rds/general/user/nbourke/home/templates/MNI152_T1_2mm_brain_pve_1_mask.nii.gz /rds/general/user/nbourke/home/projects/squeezy/scripts/design/basic.mat /rds/general/user/nbourke/home/projects/squeezy/scripts/design/contrast.con 0 0 5000 /rds/general/user/nbourke/ephemeral/mica/caudalACC/test/randomise_output/dim00 -in-step1 ` cat /rds/general/us

### Extracting mean timeseries for each network component 

Need to save files with id to run meants 

In [ ]:
# %%bash -s "$setup"
# export setup=$1;
# source $setup
# #echo $workingDir
# workingDir=/rds/general/user/nbourke/home/projects
# #------------------

# mkdir ${workingDir}/squeezy/derivatives/socRest
# data=${workingDir}/squeezy/derivatives/controlTestSet/
# for subj in `ls $data`; 
# do
#    cp ${data}/${subj}/func/rest/postICA.feat/filtered_func_data_clean_standard.nii.gz ${workingDir}/squeezy/derivatives/socRest/${subj}_ffdcs.nii.gz
# done


In [ ]:
# %%bash -s "$setup"
# export setup=$1;
# source $setup
# #echo $workingDir
# workingDir=/rds/general/user/nbourke/home/projects

# module load fsl
# # -- Specify Input --#

# # Options
# input=${workingDir}/squeezy/scripts/socSubj.txt
# brain_mask=${templates}/MNI152_T1_2mm_brain_pve_1_mask.nii.gz # GM masking

# # Mean time series extraction for network masks
# for subj in `cat $input`;
# do
#     sub=`basename ${subj}`
# #     NAME=${sub%/*/*/*/*}
# #     id=${NAME##*/}

#     for dim in 0 2 4 6 7 8 9;
#     do
#         HCP_mask=$workingDir/micaOutput/caudalACC/networkMasks/IC_0${dim}_maks.nii.gz
#         output=${workingDir}/squeezy/derivatives/mts/caudalACC/IC_0${dim}
#         mkdir -p ${output}
#         #fslmeants -i ${subj} -o ${output}/mts_IC_0${dim}_${sub}.txt -m ${HCP_mask} -v
#         fslmaths ${subj} -Tmean -mas ${HCP_mask} ${output}/Tmean_IC_0${dim}_${sub}.txt 
#     done
# done

-------------
# Notes

### Pre-processing resting state data

* The Raw DREAM data was taken to run melodic on. To be consistent with the previous processed data, the .fsf file created from the melodic gui was taken from another participant and the ID changed.  
* It is advisable to run the three different steps independently to make sure it doesn’t fail at any point. 
* ICA AROMA is an alternative to fix
* The new cluster has been a pain in the hole and the .fsf files had to be recreated with the new version of fsl. 
* Fix requires r and matlab modules to be loaded on the new cluster.



#### 1. feat was then fun on these three participants: (produces filtered_func.nii)

An example.fsf file needs to be created. A template is included at the bottom. The paths which are hardcoded into it need to be changed to suit the correct paths. 

In [ ]:
%shell

# Variables
example=$workingDir/DREAM_example.fsf
outputDir=$workingDir/${projectLabel}

# /group/DREAM/REST/REST_PAT/REST_067_v1/REST_067_v1.nii.gz

subj="067_v1 068_v1  069_v1"

for i in $subj ;
do
    echo $i
    # REST
    echo ${example} | sed 's/XX/'REST_${i}'/g' $example > $outputDir/${i}.fsf  
    # STUCT
    echo $outputDir/${i}.fsf | sed 's/YY/'${i}'/g' $outputDir/${i}.fsf > $outputDir/rest_${i}.fsf
    # run feat
    echo "feat rest_${i}.fsf" >> ${TMPDIR}/featJobs.txt
    done

    # Run job
    hpcSubmit ${TMPDIR}/featJobs.txt 12:00:00 4 6Gb